In [ ]:
from causalnex.inference import InferenceEngine
from causalnex.network import BayesianNetwork
import xml_read
import math
import numpy as np
import pandas as pd
import warnings
from tqdm.std import *
warnings.filterwarnings('ignore')

file_name = 'causal network.xml'
graph_structure = xml_read.parse_xml_sm(file_name)
data = pd.read_csv('Structural data_coarse-grained.csv')

In [ ]:
# EI calculatioin
bn = BayesianNetwork(graph_structure)
bn = bn.fit_node_states(data)
bn_cpd = bn.fit_cpds(data, method="BayesianEstimator", bayes_prior="K2")
VARIABLE = bn_cpd.nodes
print(VARIABLE)
if __name__ == '__main__':
    EI = np.zeros((len(VARIABLE),len(VARIABLE)))
    for V in tqdm(range(len(VARIABLE))):
        one_variable = VARIABLE[:]
        variable_v = one_variable.pop(V)
        ie = InferenceEngine(bn)
        ie_zero = ie.query()
        IDICT = ie_zero[VARIABLE[V]].keys()
        MIDICT = []
        for key in IDICT:
            MIDICT.append(key)
        I0 = len(MIDICT)
        PdoA_B = 1 / I0
        try:
            for i in range(len(one_variable)):
                I1 = len(ie_zero[one_variable[i]])
                P_temp = np.zeros((I0, I1))
                U_temp = np.zeros(I1)
                for k in range(I0):
                    ie_1 = InferenceEngine(bn)
                    ie_1.do_intervention(variable_v, int(MIDICT[k]))
                    ie_11 = ie_1.query()
                    for j in range(I1):
                        P_temp[k,j]=ie_11[one_variable[i]][j]
                for j in range(I1):
                    for k in range(I0):
                        U_temp[j]=U_temp[j]+P_temp[k,j]
                    U_temp[j]=U_temp[j]*PdoA_B
                for k in range(I0):
                    part=np.zeros(I1)
                    for j in range(I1):
                        if P_temp[k,j] > 0:
                            part[j] = PdoA_B * (P_temp[k,j] * math.log(P_temp[k,j] / U_temp[j]))
                        else:
                            part[j] = 0
                        EI[V][VARIABLE.index(one_variable[i])]=EI[V][VARIABLE.index(one_variable[i])]+part[j]
        except Exception:
            pass
    print(EI)
    np.save('ei.npy', EI)
    np.savetxt('EI.csv', EI, fmt='%s', delimiter=',')